# **3089. Find Bursty Behavior**

**Table: Posts**
``` console
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| post_id     | int     |
| user_id     | int     |
| post_date   | date    |
+-------------+---------+
```
post_id is the primary key (column with unique values) for this table.

Each row of this table contains post_id, user_id, and post_date.

Write a solution to find users who demonstrate bursty behavior in their posting patterns during February 2024. Bursty behavior is defined as any period of 7 consecutive days where a user's posting frequency is at least twice to their average weekly posting frequency for February 2024.

Note: Only include the dates from February 1 to February 28 in your analysis, which means you should count February as having exactly 4 weeks.

Return the result table orderd by user_id in ascending order.

The result format is in the following example.

 

**Example:**

**Input:**

**Posts table:**
``` console
+---------+---------+------------+
| post_id | user_id | post_date  |
+---------+---------+------------+
| 1       | 1       | 2024-02-27 |
| 2       | 5       | 2024-02-06 |
| 3       | 3       | 2024-02-25 |
| 4       | 3       | 2024-02-14 |
| 5       | 3       | 2024-02-06 |
| 6       | 2       | 2024-02-25 |
+---------+---------+------------+
```
**Output:**
``` console
+---------+----------------+------------------+
| user_id | max_7day_posts | avg_weekly_posts |
+---------+----------------+------------------+
| 1       | 1              | 0.2500           |
| 2       | 1              | 0.2500           |
| 5       | 1              | 0.2500           |
+---------+----------------+------------------+
```
**Explanation:**

User 1: Made only 1 post in February, resulting in an average of 0.25 posts per week and a max of 1 post in any 7-day period.

User 2: Also made just 1 post, with the same average and max 7-day posting frequency as User 1.

User 5: Like Users 1 and 2, User 5 made only 1 post throughout February, leading to the same average and max 7-day posting metrics.

User 3: Although User 3 made more posts than the others (3 posts), they did not reach twice the average weekly posts in their consecutive 7-day window, so they are not listed in the output.

Note: Output table is ordered by user_id in ascending order.

**Solution-1:**

``` sql
WITH CTE AS (
        Select *, count(*) over(partition by user_id)/4 as avg_weekly_posts
        from Posts
        where post_date between '2024-02-01' and '2024-02-28'
),

CTE2 AS (
    Select *, count(post_id) over(partition by user_id order by post_date range between interval 6 day preceding and current row) as max_7day_posts 
    from CTE
)
select distinct user_id , max(max_7day_posts) as max_7day_posts , avg_weekly_posts 
from CTE2
where max_7day_posts >=2*avg_weekly_posts 
group by 1
order by 1

**Solution-2:**

``` sql
# Write your MySQL query statement below

with t1 as (
    select user_id, 
        count(*)over(partition by user_id order by post_date range between interval 6 day preceding and current row) as max_7day_posts, 
        count(*)over(partition by user_id)/4 as avg_weekly_posts
    from posts
    where post_date between '2024-02-01' and '2024-02-28')

select distinct *
from t1
where max_7day_posts >= 2*avg_weekly_posts 
and (user_id, max_7day_posts) in (select user_id, max(max_7day_posts) from t1 group by 1)
order by user_id